In [0]:
from google.colab import files #import utils.py
files.upload()

{}

In [0]:
#packages
!pip install tensorboardX #dependancy for Logger
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import utils
from utils import Logger
from IPython import display


torch.Size([100, 100])

In [0]:
img_size=32 # resize to 32 from 28

compose = transforms.Compose([transforms.Scale(img_size),transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])
train_loader = torch.utils.data.DataLoader(dsets.MNIST('/content', train=True, download=True,transform=compose),batch_size=100, shuffle=True)

num_batch=len(train_loader)


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:209: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [0]:
class Discriminator(nn.Module): 
  def __init__(self):
    super(Discriminator,self).__init__()
    self.hidden0=nn.Sequential(    # 32* 32 
        nn.Conv2d(1,128,kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2)
    )# 128 * 16*16
    self.hidden1=nn.Sequential(
        nn.Conv2d(128,256,kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2)
    )#256*8*8
    self.hidden2=nn.Sequential(
        nn.Conv2d(256,512,kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2)
    )#512*4*4
    self.hidden3=nn.Sequential(
        nn.Conv2d(512,1024,kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(1024),
        nn.LeakyReLU(0.2)
    )#1024 * 2 * 2
    self.outlayer=nn.Sequential(
        nn.Conv2d(1024,1,kernel_size=4,stride=2,padding=1),
        nn.Sigmoid()
    )#1*1*1
    

  def forward(self,x):
    out=self.hidden0(x)
    out=self.hidden1(out)
    out=self.hidden2(out)
    out=self.hidden3(out)
    out=self.outlayer(out)
    return out

disc_model=Discriminator()
disc_model.cuda()

Discriminator(
  (hidden0): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (hidden1): Sequential(
    (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (hidden2): Sequential(
    (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (hidden3): Sequential(
    (0): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (outlayer): Sequential(
    (0): Conv2d(102

In [0]:
class Generator(nn.Module):
  def __init__(self):
    super(Generator,self).__init__()
    in_n=100
    out_n=32*32
    self.hidden0=nn.Sequential( # 100 * 1024 (=32*32)
        nn.ConvTranspose2d(in_n,1024,kernel_size=4,stride=1,padding=0),
        nn.BatchNorm2d(1024),
        nn.LeakyReLU(0.2)
    )# 1024 * 1024
    self.hidden1=nn.Sequential(
        nn.ConvTranspose2d(1024,512,kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2)
    ) # 512 * 512
    self.hidden2=nn.Sequential(
        nn.ConvTranspose2d(512,256,kernel_size=4,stride=2,padding=1),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2)
    ) # 256 * 256

    self.outlayer=nn.Sequential(
        nn.ConvTranspose2d(256,1,kernel_size=4,stride=2,padding=1),
        nn.Tanh()
    )# 1*32

  def forward(self,x):
    out=self.hidden0(x)
    out=self.hidden1(out)
    out=self.hidden2(out)
    out=self.outlayer(out)
    return out
gen_model=Generator()
gen_model.cuda()

Generator(
  (hidden0): Sequential(
    (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (hidden1): Sequential(
    (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (hidden2): Sequential(
    (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (outlayer): Sequential(
    (0): ConvTranspose2d(256, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): Tanh()
  )
)

In [0]:

def noise(size):
  n=Variable(torch.randn(size,100).view(-1,100,1,1) ) #torch.randn returns normalized random numbers between 0 and 1 (100 is our batch size , "size" variable is our noise vector size)
  return n.cuda()
def target_zeros(size): 
  n=Variable(torch.zeros(size,1)  )
  return n.cuda()
def target_ones(size): 
  n=Variable(torch.ones(size,1)  )
  return n.cuda()


In [0]:
   
disc_optimizer=optim.Adam(disc_model.parameters(),lr=0.00002)
gen_optimizer=optim.Adam(gen_model.parameters(),lr=0.00002)
loss=nn.BCELoss() # in torch we use minimze function(even though we had to use gradient ascent), BCE loss has a negitive sign which does this job for us

In [0]:
def train_disc(optimizer,real,fake) :# the discriminator takes a real image and a fake image and predicts output
  optimizer.zero_grad()
  
  predicted_real=disc_model(real)
  error_real=loss(predicted_real,target_ones(real.size(1))) #compare with ones and calculate loss gradients then update weights
  error_real.backward()
  predicted_fake=disc_model(fake)
  error_fake=loss(predicted_fake,target_zeros(real.size(1))) # compare with zeros and calculate loss gradients then update weights
  error_fake.backward()
  optimizer.step() # update weights
  return (error_real+error_fake),predicted_real,predicted_fake # see discriminator's formula for real+fake errors

In [0]:
def train_gen(optimizer,fake_image):
  optimizer.zero_grad()  
  prediction=disc_model(fake_image)
  error=loss(prediction,target_ones(prediction.size(1)))# get max 1 s for generator
  error.backward()
  
  optimizer.step()
  return error
  

In [0]:
%xmode plain

logger = Logger(model_name='VGAN', data_name='MNIST')
epoch=200
j=0
for epochs in range(epoch):
  for nbatch,(images,_) in enumerate(train_loader):
    j=j+1
    images=(images.view(1,100,32,32))
    real_image=Variable(images.cuda())
    z=noise(real_image.size(1))
    fake_image=gen_model(z).detach()
    d_error,real,fake=train_disc(disc_optimizer,real_image,fake_image) #train discriminator
    
    fake_image=gen_model(noise(real_image.size(1)))
    g_error=train_gen(gen_optimizer,fake_image) # train generator
    logger.log(d_error, g_error, epoch, nbatch, num_batch)
    if j%1000==0:
      display.clear_output(True)
      test=gen_model(noise(16))
      test=test.view(-1,1,32,32)
      test=test.data
      logger.log_images(test.cpu(), 16,epochs, nbatch, num_batch)
      logger.display_status(epochs, epoch, nbatch, num_batch,d_error, g_error, real, fake)

Exception reporting mode: Plain


RuntimeError: ignored